그레이디언트 소실과 폭주

그레이디언트 소실(Vanishing gradient)
 - 하위층으로 진행될수록 그레이디언트가 점점 작아지는 현상
 - 하위층의 연결 가중치를 변경되지 않은 채로 둔다면 훈련이 좋은 솔루션으로 
   수렴되기 어렵다

그레이디언트 폭주(Exploding gradient)
 - 소실과 반대로 그레이디언트가 점점 증가하는 현상

소실과 폭주를 막기위한 초기화 방식
 - 글로럿 :  활성화 함수없음, 하이퍼볼릭 탄젠트, 로지스틱, 소프트맥스 ( 1 / fan.avg )
 - He : ReLU 함수와 그 변종들 ( 2 / fan.in )
 - 르쿤 : SELU ( 1 / fan.in )
 - 케라스는 기본적으로 균등분포의 글로럿 초기화 방식 사용

ReLU 함수의 변종
 - 훈련 중 가중치 합이 음수이면 ReLU 함수의 그레이디언트가 0이 되므로 경사 하강법이 더는 작동하지 않는다.
 - ReLU가 죽는 현상을 막기위해 변종된 ReLU함수가 등장
  1. LeakyReLU
    - LeakyRelu.a(z)=max(az, z)로 정의
    - 하이퍼파라미터 a가 새는(leaky)정도를 결정
    - 새는 정도란 z < 0 일 때 이 함수의 기울기이며, 일반적으로 0.01로 설정
    - a=0.2로 하는 것이 a=0.1보다 더 나은 성능을 나타낸다
  2. RReLU(Randomized leaky ReLU)
    - 훈련하는 동안 주어진 범위에서 a를 무작위로 선택하고 테스트 시에는 평균을 사용
    - 훈련 세트의 과대적합 위험을 줄이는 규제 역할도 한다
  3. PReLU(Parametric leaky ReLU)
    - a가 훈련하는 동안 학습
    - 대규모 이미지 데이터셋에서는 ReLU보다 성능이 크게 앞섰지만, 소규모 데이터셋에서는 훈련 세트에 과대적합될 위험이 있다

ELU(Exponential linear unit)
 - ELU.a(z) = z < 0 일 때 a(exp(z)-1) , z >= 0 일 때 z
 - z < 0일 때 활성화 함수의 ㅎ평균 출력이 0에 더 가까워져 그레이디언트 손실문제를 완하시켜준다.
 - 하이퍼파라미터 a는 z가 큰 음숫값일 때 ELU가 수렴할 값을 정의
 - z < 0 이어도 그레이디언트가 0이 아니므로 죽는 뉴런은 안 만들어진다.
 - a = 1이면 z = 0에서 급격히 변동하지 않아 모든 구간에서 매끄러워 경사하강법의 속도를 높여준다
 - 지수 함수를 사용하므로 계산이 느리다

SELU(Scaled ELU)
 - 스케일이 조정된 ELU 활성화 함수의 변종
 - 신경망을 만들고 모든 은닉층이 SELU 활성화 함수를  사용하며 네트워크가 자기 정규화가 된다.
 - 훈련하는 동안 각 층의 출력이 평균 0과 표준편타 1을 유지해 그레이디언트 소실과 폭주 문제를 막아준다.
 - 자기 정규화 일어나는 조건
   1. 입력 특성이 반드시 표준화되어야 한다.
   2. 모든 은닉층의 가중치는 르쿤 정규분포 초기화로 초기화
   3. 네트워크는 일렬로 쌓은 층으로 구성

심층 신경망에는 일반적으로 SELU > ELU > LeakyReLU(그외 변종) > ReLU > tanh > 로지스틱 순으로 사용
 



배치 정규화(Batch normalization)
 - 그레이디언트 손실과 폭주 문제를 해결하기위한 기법
 - 각 층에서 활성화 함수를 통과하기 전이나, 후에 모델에 연산을 추가
 - 연산은 입력을 원점에 맞추고 정규화한 다음, 각 층에서 두 개의 새로운 파라미터로 결과값의 스케일을 조정하고 이동
 - 파라미터는 스케일 조정과 이동에 사용
 - 신경망의 첫 번째 층으로 배치 정규화를 추가하면 훈련 세트의 표준화가 필요 없음
 - 훈련하는 동안 배치 정규화는 입력을 정규화한 다음 스케일 조정하고 이동시킨다.
 - 훈련이 끝난 후 전체 훈련 세트를 신경망에 통과시켜 배치 정규화층의 각 입력에 대한 평균과 표준 편차를 계산
 - 예측할 때 배치 입력 평균과 표준편차로 최종 입력 평균과 표준편차 대신 사용
 - 층의 입력 평균과 표준편차의 이동 평균을 사용해 훈련하는 동안 최종 통계를 추정
 - 가중치 초기화에 네트워크가 덜 민감해진다.
 - 큰 학습률을 사용하여 학습과정의 속도를 높일 수 있다.
 - 배치 정규화는 모델의 복잡도를 키워 실행 시간면에서 손해가 나고 신경만의 예측을 느리게 한다.
 - 훈련이 끝난 후에 이전 층과 배치 정규화 층을 합쳐 실행속도 저하를 막을 수 있다.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow import keras

In [ ]:
model = keras.models.Sequential([
                                 keras.layers.Flatten(input_shape=[28, 28]),
                                 keras.layers.BatchNormalization(),
                                 keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
                                 keras.layers.BatchNormalization(),
                                 keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
                                 keras.layers.BatchNormalization(),
                                 keras.layers.Dense(10, activation="softmax")
])

In [ ]:
model.summary()

# non-trainable params : 역전파로 학습되지 않는 파라미터 수 (평균과 표준편차 파라미터)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 784)               3136      
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
batch_normalization_1 (Batch (None, 300)               1200      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
batch_normalization_2 (Batch (None, 100)               400       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1

In [ ]:
[(var.name, var.trainable) for var in model.layers[1].variables] 
# 두 개는 역전파로 훈련되고 두개는 훈련되지 않았다

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

In [ ]:
# 훈련하는 중 매 반복마다 케라스에서 호출될 두 개의 연산이 함께 생성
# 이 연산이 이동 평균을 업데이트
# 텐서플로 백엔드를 사용하므로 이 연산은 텐서플로 연산입니다
model.layers[1].updates

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


[]

In [ ]:
# 활성화 함수 전에 배치 정규화 층을 추가하려면 은닉층에서 호라성화 함수를 지정하지 말고 배치 정규화 층 뒤에 별도의 층으로 추가
# 배치 정규화 층은 입력마다 이동 파라미터를 포함하기 때문에 이전 층에서 편향을 뺄 수 있다.(use_bias=False)
model = keras.models.Sequential([
                                 keras.layers.Flatten(input_shape=[28, 28]),
                                 keras.layers.BatchNormalization(),
                                 keras.layers.Dense(300, kernel_initializer="he_normal", use_bias=False),
                                 keras.layers.BatchNormalization(),
                                 keras.layers.Activation("elu"),
                                 keras.layers.Dense(100, kernel_initializer="he_normal", use_bias=False),
                                 keras.layers.BatchNormalization(),
                                 keras.layers.Activation("elu"),
                                 keras.layers.Dense(10, activation="softmax")
])

그레이디언트 클리핑
 - 그레이디언트 폭주 문제를 완하하는 방법
 - 역전파될 때 일정 임곗값을 넘어서지 못하게 그레이디언트를 잘라내는 방식
 - 그레이디언트 벡터의 모든 원소를 -1.0과 1.0  사이로 클리핑
 - 손실의 모든 편미분 값을 -1.0에서 1.0으로 잘라낸다

In [ ]:
# 케라스에서 그레이디언트 클리핑 구현
# optimizer 생성 시 clipvalue와 clipnorm 매개변수를 지정
optimizer = keras.optimizers.SGD(clipvalue=1.0)
model.compile(loss="mse", optimizer=optimizer)

사전훈련된 층 재사용하기
 - 전이 학습(transfer learning) : 해결 하려는 것과 비슷한 유형의 문제를 처리한 신경망이 있으면 그 신경망의 하위층을 재사용
 - 이 방법은 훈련 속도를 크게 높이고 필요한 훈련 데이터도 크게 줄인다.
 - 먼저 재사용하는 층은 모두 동결한다.(SGD로 가중치가 바뀌지 않도록 훈련되지 않는 가중치로 만든다.)
 - 모델을 훈련하고 성능을 평가
 - 맨 위에 있는 한두개의 은닉층의 동결을 해제하고 역전파를 통해 가중치를 조정하여 성능 확인
 - 훈련 데이터가 많은수록 많은 층의 동결을 해제할 수 있다.
 - 재사용 층의 동결을 해제 시 학습률을 작게한다.
 - 성능이 낮고 훈련 데이터가 적다면 상위 은닉층(들)을 제거하고 남은 은닉층을 다시 동결
 - 재사용할 은닉층의 적절한 개수를 찾을때 까지 반복

In [ ]:
# model_A와 model_B_on_A는 일부 층을 공유
# model_A_on_B 훈련 시 model_A도 영향을 받는다
model_A = keras.models.load_model("my_model_A.h5")
model_B_on_A = keras.models.Sequential(model_A.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

# 영향 받는 거를 원치 않을 시 model_A를 클론하여 사용
model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())

In [ ]:
# 새로운 출력층이 랜덤하게 초기화되어 있으므로 큰 오차를 만들 것이다
# 피하기 위해 처음 몇번의 에포크 동안 재사용된 층을 동결하고 새로운 층에게 적절한 가중치를 학습할 시간을 준다
# 모든 층의 trainalbe 속성을 False로 지정하고 컴파일
for layer in model_B_on_A.layers[:-1]:
  layer.trainable = False

model_B_on_A.compile(loss="binary_crossentropy", optimizer="sgd",
                     metrics=["accuracy"])

In [ ]:
# 재사용된 층의 동결을 해제하고 훈련
# 일반적으로 재사용된 층의 동결을 해제한 후에 학습률을 낮추는 것이 좋다
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4, validation_data=(X_valid_B, y_valid_B))

for layer in model_B_on_A.layers[:-1]:
  layer.trainable = False

optimizer = keras.optimizers.SGD(lr=1e-4) # 기본 학습률 1e-2
model_B_on_A.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16, validation_data=(X_valid_B, y_valid_B))

In [ ]:
model_A_on_B.evaluate(X_text_B, y_text_B)

고속 옵티마이저

In [ ]:
# 모멘텀 최적화
# 이전 그레이디언트가 얼마였는지를 중요하게 여김
# 매 반복에서 현재 그레이디언트를(학습률 곱한 후) 모멘텀 벡터m에 더하고 이 값을 빼는 방식으로 가중치를 갱신
# 모멘텀 하이퍼파라미터b는 0~1사이의 값을 갖고 기본적으로 0.9를 사용
# 지역 최적점을 건너뛰도록 하는데 도움
optimizer = keras.optimizers.SGD(lr=0.001, momentum=0.9)

In [ ]:
# 네스테로프 가속 경사(Nesterov accelerated gradient)
# 모멘텀의 현재 위치에서 +bm만큼 이동시켜 그레이디언트를 계산
# 이동으로 인해 네스테로프 업데이가 최적값에 더 가깝다.
optimizer = keras.optimizers.SGD(lr=0.001, momentum=0.9, nesterov=True)

In [ ]:
# AdaGrad
# 가장 가파른 차원을 따라 그레이디엉ㄴ트 벡터의 스케일을 감소시키는 방식
# 학습률을 감소시키지만 경사가 완만한 차원보다 가파른 차원에 대해 더 빠르게 감소하고 이를 적응적 학습률이라 부른다.
# 간단한 2차 방정식에서 잘 작동하지만 신경망 훈련 시 너무 일직 멈추는 경우가 발생
# 심층 신경망에서는 사용을 안하는 것이 좋다

In [ ]:
# RMSProp
# AdaGrad의 문제점을 훈련 시작부터의 모든 그레이디언트가 아닌 가장 최근 반복에서 비롯된 그레이디언트만 누적함으로서 해결
# 하이퍼파라미터 감쇠율이 추가되었고 기본값으로 0.9를 사용한다
optimizer = keras.optimizers.RMSprop(lr=0.001, rho=0.9)

In [ ]:
# Adam
# 적응적 모멘트 추정(Adaptive moment estimation)
# 모멘텀 최적화와 RMSProp의 아이디어를 합친 것
# 모멘텀 최적화처럼 지난 그레이디언트의 지수 감소 평균을 따르고 
# RMSProp처럼 지난 그레이디언트 제곱의 지수 감소된 평균을 따른다.
# 모멘텀 감쇠 하이퍼파라미터 b1은 보통 0.9로 초기화,
# 스케일 감쇠 하이퍼파라미터 b2는 0.999,
# 안정된 계산을 위한 e는 보통 10e-7같은 아주 작은수로 초기화
optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999)

In [ ]:
# AdaMax
# Adam의 L2노름을 L무한 노름으로 바꾼다
# AdaMax가 Adam보다 더 안정적이지만 실제로  데이터셋에 따라 다르다

# Nadam
# Adam 옵티마이저에 네스테로프 기법을 더한 것
# 종종 Adam보다 빠르게 수렴

학습률 스케쥴링
 - 훈련하는 동안 학습률을 감소시키는 전략
 - 학습률을 너무 크게 잡으면 훈련이 실제로 발산한다.
 - 너무 작게 잡으면 최적점에 수렴하지만 시간이 매우 오래걸린다.
 - 조금 높게 잡으면 처음에는 매우 빠르게 진행하지만 최적점 근처에서는 요동이 심해져 수렴하지 못한다.
 - 간단한 방법으로는 매우 작은 값에서 매우 큰 값까지 지수적으로 학습률을 증가시키면서 모델 훈련을 수백 번 반복하여 좋은 학습률을 찾는 방법
 - 또는 큰 학습률로 시작하고 학습 속도가 느려질 때 학습률을 낮추면 최적의 고정 학습률보다 좋은 솔루션을 더 빨리 발견할 수 있다.

In [ ]:
# 거듭제곱 기반 스케줄링(Power Scheduling)
# decay 매개변수만 지정
# decay는 s(학습률을 나누기 위해 수행할 스텝 수)의 역수
optimizer = keras.optimizers.SGD(lr=0.01, decay=1e-4)

In [ ]:
# 지수 기반 스케줄링(Exponential Scheduling)

def exponential_decay(lr0, s):
  def exponential_decay_fn(epoch):
    return lr0 * 0.1**(epoch / s)
  return exponential_decay_fn

exponential_decay_fn = exponential_decay(lr0=0.01, s=20)

lr_shceduler = keras.callbacks.LearningRateScheduler(exponential_decay_fn)
history = model.fit(X_train_scaled, y_train, [...], callbacks=[lr_scheduler])

# 스케줄 중에 하나를 사용해 학습률을  정희하고 이 학습률을 옵티마이저에 전달
# 해당 방법으로 구현 시 에포크가 아니라 매 스텝마다 학습률을 업데이트
s = 20 * len(X_train) // 32 # 20번 에포크에 담긴 전체 스텝 수(배치크기 = 32)
learning_rate = keras.optimizers.schedules.ExponentialDecay(0.01, s, 0.1)
optimizer = keras.optimizers.SGD(learning_rate)

In [ ]:
# 구간별 고정 스케줄링(Piecewise Constant Scheduling)
def piecewise_constant_fn(epoch):
  if epoch < 5:
    return 0.01
  elif epoch < 15:
    return 0.005
  elif:
    return 0.001

In [ ]:
# 성능 기반 스케줄링(Performance Scheduling)
lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)

In [ ]:
# 1사이클 스케줄링(1cycle Scheduling)

규제를 사용해 과대적합 피하기

In [ ]:
# L1과 L2 규제
# 신경망의 연결 가중치를 제한하기 위해 L2 규제를 사용하거나
# 희소 모델을 만들기 위해 L1 규제를 사용할 수 있다.

# 연결 가중치에 규제강도 0.01을 사용하여 L2 규제 적용
layer = keras.layers.Dense(100, activation="elu",
                           kernel_initializer="he_normal",
                           kernel_regularizer=keras.regularizers.l2(0.01))


# 동일한 활성화 함수, 초기화 전략이나 동일한 규제로 매개변수 값을 반복하는 경우 
# 파이썬의 functools.partial() 함수를 사용하여 기본 매개변수 값을 사용
from functools import partial

RegularizedDense = partial(keras.layers.Dense,
                           activation="elu",
                           kernel_initializer="he_normal",
                           kernel_regularizer=keras.regularizers.l2(0.01))

model = keras.models.Sequentia([
                                keras.layers.Flatten(input_shape=[28,28]),
                                RegularizedDense(300),
                                RegularizedDense(100),
                                RegularizedDense(10, activation="softmax",
                                                 kernel_initializer="glorot_uniform")
])

In [ ]:
# 드롭아웃
# 매 훈련 습텝에서 각 뉴런은 임시적으로 드롭아웃될 확률 p를 가진다.
# 이번 훈련 스텝에는 완전히 무시되지만 다음 스템에는 활성화될 수 있다.
# 하이퍼파라미터 p를 드롭아웃 비율이라고하고 보통 10~50% 사이를 지정
# 훈련이 끝난 후에는 뉴런에 드롭아웃을 적용하지 않는다.
# 테스트하는 동안에는 하나의 뉴런이 훈련 때보다 (평균적으로) 두 배 많은
# 입력 뉴런과 연결된다.
# 훈련이 끝난 뒤 각 입력의 연결 가중치에 보존확률(1-p)를 곱해야 한다.
model = keras.models.Sequential([
                                 keras.layers.Flatten(input_shape=[28, 28]),
                                 keras.layers.Dropout(rate=0.2),
                                 keras.layers.Dense(300, activation="elu",
                                                    kernel_initializer="he_normal"),
                                 keras.layers.Droupout(rate=0.2),
                                 keras.layers.Dense(100, activation="elu",
                                                    kernel_initializer="he_normal"),
                                 keras.layers.Dropout(rate=0.2),
                                 keras.layers.Dense(10, activation="softmax"))
])

In [ ]:
# 몬테 카를로 드롭아웃
# 드롭아웃 네트워크와 근사 베이즈 추론 사이에 깊은 고나련성을 정립
# 훈련된 드롭아웃 모델을 재훈련하거나 전혀 수정하지 않고 성능을 크게 향상 시킬수 있다.

# training=True로 지정하여 Droupout 층을 활성화하고 테스트 세트에서 100번의 예측을 만들어 쌓는다.
y_probas = np.stack([model(X_test_scaled, training=True) for sample in range(100)])
y_proba = y_probas.mean(axis=0)

# 모델이 훈련하는 동안 다르게 작동하는(BatchNormalization층과 같은) 층을 가지고 있다면
# 앞에서와 같이 훈련 모드를 강제로 설정해서는 안된다.
# 대신 Droupout층을 다음과 같은 MCDropout 클래스로 바꿔준다.

class MCDropout(keras.layers.Dropout):
  def call(self, inputs):
    return super().call(inputs, training=True)

In [ ]:
# 맥스-노름 규제
# 각각의 뉴런에 대해 입력의 연결 가중치 W가 L2노름(W) <= r(맥스-노름 하이퍼파라미터)
# 매 훈련 스텝이 끝나고 L2(W)를 계산하고 필요하면 W의 스케일 조정
# r을 줄이면 규제의 양이 증가하여 과대적합을 감소시키는데 도움된다.
# 불안정한 그레이디언트 문제를 완하하는데 도움을 준다.

keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal",
                   kernel_constraint=keras.constraints.max_norm(1.)))